In [212]:
from __future__ import print_function
import numpy as np
import os
import math
import collections
import json

In [2]:
import data_helper as dh

## Take Mohamed's tokenized Wikipedia sentences and create data files

1. Generate a vocabulary
2. Specify it to a particular size
3. Replace oovs
4. Parse with spacy
5. Create sdp from each pair of noun phrase heads in a sentence
   - Include the depedency it is a head of
6. Replace sequence pairs with indices
7. Write out vocab files and data files, line by line

In [100]:
def noun_chunk_to_head_noun(chunk):
    """Given a chunk, find the noun who's head is outside the chunk. This is the head noun"""
    chunk_set = set(list(chunk))
    for token in chunk:
        if token.head not in chunk_set:
            return token
    print("No head noun found in chunk... %r" % chunk.text)
    return None

In [101]:
def sentence_to_chunk_pairs(sentence):
    """Iterate over  sentence generating n choose 2 noun phrase heads"""
    chunk_pairs = []
    noun_chunks = list(sentence.noun_chunks)
    for i, chunk1 in enumerate(noun_chunks[:-1]):
        head1 = noun_chunk_to_head_noun(chunk1)
        if not head1:
            continue # don't let bad noun chunks in
        for chunk2 in noun_chunks[i+1:]:
            head2 = noun_chunk_to_head_noun(chunk2)
            if not head2:
                continue # don't let bad noun chunks in
            chunk_pairs.append((head1, head2))
    return chunk_pairs

# test = sentences[1]
# chunks = list(test.noun_chunks)
# pairs = sentence_to_chunk_pairs(test)
# print(test)
# print(chunks)
# print(len(pairs), pairs)
# assert len(pairs) == math.factorial(len(chunks))/float(2*math.factorial(len(chunks) - 2))

In [102]:
def dependency_path_to_root(token):
    """Traverse up the dependency tree. Include the token we are tracing"""
    dep_path = [token]
    while token.head is not token:
        dep_path.append(token.head)
        token = token.head
    # dep_path.append(token.head) # add the root node
    return dep_path

def find_common_ancestor(e1_path, e2_path):
    """Loop through both dep paths and return common ancestor"""
    for t1 in e1_path:
        for t2 in e2_path:
            if t1.idx ==  t2.idx:
                return t1
    return None

In [103]:
test = dh.nlp(u'As a composer , Artur Cimirro is strongly influenced by the composer / pianists of different languages such as Franz Liszt , Leopold Godowsky , Ferrucio Busoni , Kaikhosru Shapurji Sorabji .')
# test = dh.nlp(u"This sentence has multiple nouns, like apple, orange, and banana")
print(list(test.noun_chunks))
print([noun_chunk_to_head_noun(chunk) for chunk in test.noun_chunks])
print(sentence_to_chunk_pairs(test))


[a composer, Artur Cimirro, the composer, different languages, Franz Liszt, Leopold Godowsky, Ferrucio Busoni]
[composer , Cimirro , composer , languages , Liszt , Godowsky , Busoni ]
[(composer , Cimirro ), (composer , composer ), (composer , languages ), (composer , Liszt ), (composer , Godowsky ), (composer , Busoni ), (Cimirro , composer ), (Cimirro , languages ), (Cimirro , Liszt ), (Cimirro , Godowsky ), (Cimirro , Busoni ), (composer , languages ), (composer , Liszt ), (composer , Godowsky ), (composer , Busoni ), (languages , Liszt ), (languages , Godowsky ), (languages , Busoni ), (Liszt , Godowsky ), (Liszt , Busoni ), (Godowsky , Busoni )]


In [104]:
def sentence_to_sdps(sentence, min_len=1):
    """Takes sentence and returns all shortest dependency paths (SDP) between pairs of noun phrase heads in a sentence
    
    Args:
        sentence: a spacy Sentence
        min_len (opt): the minimum number of words along the path (not including endpoints)
    
    Returns:
        sdps: a dict with `path` and `target` fields
                where `path` is a list of (word, dep) tuples (where dep is the dependency that word is the head of)
                and `target` is the pair of head nouns at the endpoints of the path
                
    Notes:
        There are three cases of SDPs: 
        (1) there is not dependency path between X and Y. We obviously skip these
        (2) one nominal lies on the path to root of the other (wlog X <- ... <- Y <- ...)
            In this case we will lose one dependency, the one where X is the tail.
                                                                            | 
                                                                            v
        (3) the nominals have common ancestor in the tree (wlog X <- ... <- Z -> ... -> Y)
            In this case we will lose two dependencies, those involving X and Y.
    """
    noun_pairs = sentence_to_chunk_pairs(sentence)
    for X, Y in noun_pairs:
        X_path = dependency_path_to_root(X)
        Y_path = dependency_path_to_root(Y)
        common = find_common_ancestor(X_path, Y_path) # need nouns to find case (2)
        # now we don't want nouns for assembly
        X_path = X_path[1:]
        Y_path = Y_path[1:]
        # CASE (1)
        if not common:
            print("Bad SDP: skipping")
            continue
            
        # CASE (2)
        elif X is common:
#             print("X %r is common" % common)
            sdp = []
            for token in Y_path: # looks like Y <- (...) <- X <- ...
                if token is common: # stop before X
                    break
                sdp.append((token.text, token.dep_))
            sdp = list(reversed(sdp)) # flip to get -> X -> (...) -> Y
        elif Y is common:
#             print("Y %r is common" % common)
            sdp = []
            for token in X_path: # looks like X <- ... <- Y 
                if token is common: # stop before Y
                      break
                sdp.append((token.text, token.dep_))
    
        # CASE (3)
        else:
#             print("Z %r is common" % common)
            sdp = []
            for token in (X_path): # looks like X <- (... <- Z <-) ...
                sdp.append((token.text, token.dep_))
                if token is common: # keep Z this time
                    break
            ysdp = [] # need to keep track of seperate, then will reverse and extend later
            for token in Y_path: # looks like (Y <- ... <-) Z <- ... 
                if token is common:
                    break
                ysdp.append((token.text, token.dep_))
            sdp.extend(list(reversed(ysdp))) # looks like X <- (... <- Z -> ... ) -> Y
            
        if len(sdp) < min_len:
            continue # skip ones that are too short
        yield {'path': sdp, 'target':(X.text, Y.text)}
    
# test = dh.nlp(u'As a composer , Artur Cimirro is strongly influenced by the composer / pianists of different languages such as Franz Liszt , Leopold Godowsky , Ferrucio Busoni , Kaikhosru Shapurji Sorabji .')

# for sdp in sentence_to_sdps(test):
#     print(sdp['target'], sdp['path'])
#     print('-'*80)

In [215]:
def create_vocab_from_data(sentences, vocab_limit=None, 
                           min_count=None, dep=False, 
                           filter_oov=False, print_oov=False,
                           oov_count=1):
    """Create a vocab index, inverse index, and unigram distribution over tokens from a list of spacy sentences
    
    if `dep`=True, return the dependencies instead of the tokens"""
    counts = collections.Counter()
    for sentence in sentences:
        for token in sentence:
            if dep:
                counts[token.dep_] += 1
            else:
                if filter_oov and not token.is_oov and token.text not in [u' ', u'\n\n']:
                    counts[token.text] += 1
                elif not filter_oov and token.text not in [u' ', u'\n\n']:
                    counts[token.text] += 1
                elif print_oov:
                    print("Token %r is oov" % token.text)
    
    counts = counts.most_common()
#     print(counts)
    if not (vocab_limit or min_count):
        vocab_limit = len(counts)
    elif vocab_limit > len(counts):
        print("Your vocab limit %i was bigger than the number of token types, now it's %i" 
              % (vocab_limit, len(counts)))
        vocab_limit = len(counts)
    elif min_count:
        # get first index of an element that doesn't meet the requency constraint
        vocab_limit = len(counts) # never found something too small
        for i, count in enumerate(map(lambda x:x[1], counts)):
            if count < min_count:
                vocab_limit = i
                break
    
#     print(len(counts), vocab_limit)
    # create the vocab in most common order
    # include an <OOV> token and make it's count the sum of all elements that didn't make the cut
    vocab = [ x[0] for x in counts][:vocab_limit] + [u'<OOV>']
    if not oov_count and vocab_limit < len(vocab): # if we didn't specify a psuedocount, take the real one... probably a bad idea
        oov_count = sum(map(lambda x:x[1], counts[vocab_limit:]))
    freqs = [ x[1] for x in counts ][:vocab_limit] + [oov_count]
    # calculate the empirical distribution
    unigram_distribution = list(np.array(freqs) / np.sum(freqs, dtype=np.float32))
    # create index and inverted index
    vocab2int = { token:i for (i, token) in enumerate(vocab) }
    int2vocab = { i:token for (token, i) in vocab2int.items() }

    return vocab, vocab2int, int2vocab, unigram_distribution

In [218]:
def vocab2idx(token, vocab2int):
    if token in vocab2int:
        return vocab2int[token]
    else:
        return len(vocab2int)-1 # OOV conversion

In [229]:
FLAGS = {
    'num_sentences': 100, # max is 31661479
    'vocab_limit':None,
    'min_count':3,
    'data_dir':'data/',
    'sentence_file':'en.tok.txt',
    'out_prefix':'dep_rnn_'
}

In [207]:
sentences = []
for i, line in enumerate(open(os.path.join(FLAGS['data_dir'], FLAGS['sentence_file']), 'r')):
    if i > FLAGS['num_sentences']:
        break
    sentences.append(dh.nlp(unicode(line.strip())))

In [208]:
vocab, vocab2int, int2vocab, vocab_dist = create_vocab_from_data(sentences,
                                                                 vocab_limit=FLAGS['vocab_limit'],
                                                                 min_count=FLAGS['min_count'],
                                                                 dep=False,
                                                                 oov_count=1)
dep_vocab, dep2int, int2dep, dep_dist = create_vocab_from_data(sentences,
                                                                 vocab_limit=None,
                                                                 min_count=0,
                                                                 dep=True,
                                                                 oov_count=1)

In [230]:
with open(FLAGS['out_prefix'] + str(FLAGS['num_sentences']), 'w') as outfile:
    for sentence in sentences:
        for sdp in sentence_to_sdps(sentence):
            # convert from tokens to indices
            sdp['path'] = [ (vocab2idx(x[0], vocab2int), vocab2idx(x[1], dep2int)) for x in sdp['path'] ]
            sdp['target'] = [ (vocab2idx(sdp['target'][0], vocab2int), vocab2idx(sdp['target'][1], vocab2int)) ]
            # write out the dict as json line
            outfile.write(json.dumps(sdp) + '\n')

In [231]:
with open(os.path.join(FLAGS['data_dir'], FLAGS['out_prefix'] + str(FLAGS['num_sentences'])+'_vocab'), 'w') as outfile:
    for term in zip(vocab, vocab_dist):
        outfile.write(json.dumps(term)+'\n')

In [237]:
with open(os.path.join(FLAGS['data_dir'], FLAGS['out_prefix'] + str(FLAGS['num_sentences'])+'_dep'), 'w') as outfile:
    for term in zip(dep_vocab, dep_dist):
        outfile.write(json.dumps(term)+'\n')

In [222]:
ls data

dep_rnn100                            en.tok.txt.gz*
en.tok.txt*                           enwiki-latest-pages-articles.xml.bz2


In [238]:
data = []
with open('data/dep_rnn_100_dep', 'r') as f:
    for line in f:
        data.append(json.loads(line))

In [239]:
print(*data)

[u'pobj', 0.1259320629660315] [u'prep', 0.12178956089478045] [u'punct', 0.11019055509527755] [u'det', 0.09527754763877382] [u'amod', 0.07497928748964375] [u'compound', 0.0687655343827672] [u'nsubj', 0.04225352112676056] [u'ROOT', 0.04183927091963546] [u'conj', 0.04101077050538525] [u'advmod', 0.03603976801988401] [u'cc', 0.03231151615575808] [u'dobj', 0.026097763048881523] [u'aux', 0.02112676056338028] [u'poss', 0.016984258492129246] [u'auxpass', 0.015741507870753936] [u'nummod', 0.015327257663628831] [u'nsubjpass', 0.014084507042253521] [u'attr', 0.013256006628003313] [u'advcl', 0.012013256006628004] [u'appos', 0.010356255178127589] [u'relcl', 0.008285004142502071] [u'agent', 0.007042253521126761] [u'acl', 0.007042253521126761] [u'xcomp', 0.00579950289975145] [u'npadvmod', 0.004971002485501243] [u'case', 0.004971002485501243] [u'pcomp', 0.004142502071251036] [u'acomp', 0.003728251864125932] [u'ccomp', 0.0033140016570008283] [u'nmod', 0.002899751449875725] [u'oprd', 0.00248550124275062